In [1]:
import os, re
import numpy as np
import h5py
import pyriemann
from matplotlib import pyplot as plt

os.chdir("/Users/fadilboodoo/Desktop/Turing/M1/Cours/TER/Dataset/dsp.nosync")

def get_h5(file_dir):
    return [os.path.join(file_dir, f)
        for f in os.listdir(file_dir)
        if os.path.isfile(os.path.join(file_dir, f)) and 'h5' in f]



def load_data(data_path, mode = 'single_user', nb_frames = 1000000, channels = 4):
    
    data_o = []
    labels_o = []
    return_list = []
    
    num_channel = channels
    if mode == 'cross_user':
        sessions = [2,3,5,6,8,9,10,11,12,13]
    elif mode == 'single_user':
        sessions = [0]


    #numpy init
    if mode == 'cross_user':
        #nb_gestures*nb_sessions*nb_instances*(nb_channels*nb_frames*32*32)
        data = np.zeros((11,10,25,num_channel,nb_frames,32,32))
        # One label per frame
        labels = np.zeros((11,10,25,nb_frames))


    #get gestures files
    source_files = get_h5(data_path)

    for s in source_files:

        d = os.path.basename(s).split('_')
        gestureID, sessionID, inst = int(d[0]), int(d[1]), int(d[2].split('.')[0])

        if sessionID in set(sessions):
            #print('Generating', s)
            with h5py.File(s, 'r') as f:
                lab = f['label'][()]
                num_frames = lab.shape[0]
                for c in range(num_channel):
                    # Data and label are numpy arrays
                    data = f['ch{}'.format(c)][()].reshape(num_frames,32,32)
                    data_o.append(data)
                    labels_o.append(lab)
    return_list = [data_o,labels_o]
    return(return_list)

In [2]:
Dataset_0 = load_data("/Users/fadilboodoo/Desktop/Turing/M1/Cours/TER/Dataset/dsp.nosync")

In [181]:
#nbr_sequences = len(Dataset_0[0])
nbr_sequences = 1000
Nf = len(Dataset_0[0][0][0][0])

features = np.zeros((nbr_sequences,Nf,Nf))
labels = np.zeros(nbr_sequences)


for k in range (0, nbr_sequences):
    NT = Dataset_0[0][0].shape[0]
    matrice_conc = Dataset_0[0][0][0]

    #for i in range (1, NT):
    #    matrice_conc = np.concatenate((matrice_conc, Dataset_0[0][0][i]), axis=0, out=None)
        
    matrice_conv = pyriemann.estimation.Covariances('oas').transform(Dataset_0[0][k])
    matrice_moy_cov = pyriemann.utils.mean.mean_covariance(matrice_conv)
    
    features[k] = matrice_moy_cov
    labels[k] = Dataset_0[1][k][0]

In [ ]:
nbr_clusters = len(set(labels[0:nbr_sequences]))
Kmean = pyriemann.clustering.Kmeans(nbr_clusters).fit(features)
result = Kmean.score(features,labels)
result

In [4]:
matrice_conv = pyriemann.estimation.Covariances('oas').transform(Dataset_0[0][0])
matrice_moy_cov = pyriemann.utils.mean.mean_covariance(matrice_conv)
feature0 = matrice_moy_cov

In [7]:
matrice_conv = pyriemann.estimation.Covariances('oas').transform(Dataset_0[0][1])
matrice_moy_cov = pyriemann.utils.mean.mean_covariance(matrice_conv)
feature1 = matrice_moy_cov

In [6]:
feature0

array([[0.00023734, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00235552, 0.00310102, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00310102, 0.02136381, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00023734, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00023734,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00023734]])

In [8]:
np.insert(feature0,1,feature1)

ValueError: could not broadcast input array from shape (32,32) into shape (32)

In [9]:
feature0.shape

(32, 32)

In [10]:
test = np.array

In [11]:
np.insert(test,1,feature1)

ValueError: could not broadcast input array from shape (32,32) into shape (32)

In [12]:
newArray = np.append(feature0, feature1)

In [14]:
newArray.shape

(2048,)

In [15]:
len(Dataset_0[0])

2200